
# Exploring App Profiles on the App Store and Play Store

My aim in this project is to explore app store data using only Python (not Pandas). Simply as a way to practice pure Python programming. I've borrowed heavily from DataQuest's guided projects for this analysis. I'm using the fake premise that I'm working as data analysts for a company that builds Android and iOS mobile apps.

For the sake of the exercise, I'll focus only on free apps in English. My goal is to understand the kinds of apps that are most popular within that population and report back to the company with recommendations on the kinds of apps we should build.


# Opening and Exploring the Data
I've taken two datasets from Kaggle that have enough information for me to work with, given the above premise and are of reasonable size. 

As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play. 

I used a [data set](https://www.kaggle.com/gauthamp10/google-playstore-apps) containing data about approximately ten thousand Android apps from Google Play. And a [data set](https://www.kaggle.com/fentyforte/365k-ios-apps-dataset) containing data about approximately seven thousand iOS apps from the App Store. 


In [1]:
from csv import reader

#ios data
opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

#android data
opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android = list(read_file)

android_header = android[0]
android = android[1:]


In [2]:
#function used to explore the data set
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [3]:
explore_data(ios, 4, 7, True)

['5', '282935706', 'Bible', '92774400', 'USD', '0', '985920', '5320', '4.5', '5', '7.5.1', '4+', 'Reference', '37', '5', '45', '1']


['6', '283619399', 'Shanghai Mahjong', '10485713', 'USD', '0.99', '8253', '5516', '4', '4', '1.8', '4+', 'Games', '47', '5', '1', '1']


['7', '283646709', 'PayPal - Send and request money safely', '227795968', 'USD', '0', '119487', '879', '4', '4.5', '6.12.0', '4+', 'Finance', '37', '0', '19', '1']


Number of rows: 7197
Number of columns: 17


In [4]:
explore_data(android, 4, 7, True)

['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


['Smoke Effect Photo Maker - Smoke Editor', 'ART_AND_DESIGN', '3.8', '178', '19M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'April 26, 2018', '1.1', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


In [5]:
print(ios_header)
print('\n')
print(android_header)

['', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


### Deleting Wrong Data
I know from using the Dataquest guide that the Google Play data set has a single row that is missing data. I'll just go ahead and remove that row directly.

In [7]:
del android[10472]
#don't run more than once

As part of the data cleaning process, let's check for duplicate entries (a common occurance). 

In [80]:
#check dupes android

duplicate_apps_android = []
unique_apps_android = []


    
for app in android:
    name = app[0] 
    if name in unique_apps_android:
        duplicate_apps_android.append(name)
        
    else:
        unique_apps_android.append(name)

print("Number of duplicate Android apps: ", len(duplicate_apps_android))
print('\n')
print("Number of unique Android apps: ", len(unique_apps_android))

Number of duplicate Android apps:  1181


Number of unique Android apps:  9659


In [81]:
#check dupes ios

duplicate_apps_ios = []
unique_apps_ios = []


    
for app in ios:
    name = app[10] 
    if name in unique_apps_ios:
        duplicate_apps_ios.append(name)
        
    else:
        unique_apps_ios.append(name)

print("Number of duplicate iOS apps: ", len(duplicate_apps_ios))
print('\n')
print("Number of unique iOS apps: ", len(unique_apps_ios))

Number of duplicate iOS apps:  5607


Number of unique iOS apps:  1590


After looking at specific instances of duplicates, e.g.

```
 for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)
```

it looks like the duplicate rows differ in their review counts. I'll assume that the row with the highest review counts is the most recent and will remove all others by using a dictionary where the key is the app name and the value is the high number of reviews of that particular app.


In [9]:
reviews_max = {}

for row in android:
    name = row[0]
    n_reviews = float(row[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
    
print(len(reviews_max))

9659


In [10]:
print('Expected length:', len(android) - 1181)
print('Actual length:', len(reviews_max))

Expected length: 9659
Actual length: 9659


In [11]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)
        

In [12]:
explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


# Removing Non-English Apps
### Part One
When exploring the data set I noticed that there are some apps that are clearly not English-language apps. As a good proxy solution, I'm going to use the ASCII character encodings to look for non-English characters and remove apps with more than three such characters in their titles. The threshold is so that I don't remove apps with, for example, an emoji in the title. Not perfect but it'll do.

In [13]:


def is_english(string):
    
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
        
    if non_ascii > 3:
        return False
    
    else:
        return True

            

In [82]:
#testing the function
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
True
True
False


In [15]:
#cleaning up each list
android_english = []
ios_english = []

for app in android_clean:
    name = app[0]
    if is_english(name):
        android_english.append(app) 

        
for app in ios:
    name = app[2]
    if is_english(name):
        ios_english.append(app)
        

        
print(len(android_english))
print(len(ios_english))



9614
6183


In [83]:
#exploring data after removing duplicates & non-English apps
explore_data(android_english, 0, 3, True)
print('\n')
explore_data(ios_english, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13


['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1']


['2', '281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2', '4+', 'Productivity', '37', '5', '23', '1']


['3', '281940292', 'WeatherBug - Local Weather, Radar, Maps, Alerts', '100524032', 'USD', '0', '188

# Isolating the Free Apps
Self-explanatory

In [27]:
android_final = []
ios_final = []


for app in ios_english:
    if app[5] == '0':
        ios_final.append(app)
        
for app in android_english:
    if app[6] == 'Free':
        android_final.append(app)
        
        
print(len(android_final))        
print(len(ios_final))        
        

8863
3222




# Most Common Apps by Genre
In order to find the most common apps by genre, I built a function to generate a frequency table that shows the percentage of total games that are in a particular category. 

In [42]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
            
            
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage
        
    return table_percentages

            

In [44]:
freq_table(ios_final, -5)

{'Productivity': 1.7380509000620732,
 'Weather': 0.8690254500310366,
 'Shopping': 2.60707635009311,
 'Reference': 0.5586592178770949,
 'Finance': 1.1173184357541899,
 'Music': 2.0484171322160147,
 'Utilities': 2.5139664804469275,
 'Travel': 1.2414649286157666,
 'Social Networking': 3.2898820608317814,
 'Sports': 2.1415270018621975,
 'Health & Fitness': 2.0173805090006205,
 'Games': 58.16263190564867,
 'Food & Drink': 0.8069522036002483,
 'News': 1.3345747982619491,
 'Book': 0.4345127250155183,
 'Photo & Video': 4.9658597144630665,
 'Entertainment': 7.883302296710118,
 'Business': 0.5276225946617008,
 'Lifestyle': 1.5828677839851024,
 'Education': 3.662321539416512,
 'Navigation': 0.186219739292365,
 'Medical': 0.186219739292365,
 'Catalogs': 0.12414649286157665}

In [45]:
freq_table(android_final, 1) #category

{'ART_AND_DESIGN': 0.6431230960171499,
 'AUTO_AND_VEHICLES': 0.9251946293580051,
 'BEAUTY': 0.5979916506826132,
 'BOOKS_AND_REFERENCE': 2.1437436533904997,
 'BUSINESS': 4.592124562789123,
 'COMICS': 0.6205573733498815,
 'COMMUNICATION': 3.2381812027530184,
 'DATING': 1.8616721200496444,
 'EDUCATION': 1.1621347173643235,
 'ENTERTAINMENT': 0.9590432133589079,
 'EVENTS': 0.7108202640189552,
 'FINANCE': 3.7007785174320205,
 'FOOD_AND_DRINK': 1.241114746699763,
 'HEALTH_AND_FITNESS': 3.0802211440821394,
 'HOUSE_AND_HOME': 0.8236488773552973,
 'LIBRARIES_AND_DEMO': 0.9364774906916393,
 'LIFESTYLE': 3.9038700214374367,
 'GAME': 9.725826469592688,
 'FAMILY': 18.898792733837304,
 'MEDICAL': 3.5315355974275078,
 'SOCIAL': 2.6627552747376737,
 'SHOPPING': 2.245289405393208,
 'PHOTOGRAPHY': 2.944826808078529,
 'SPORTS': 3.396141261423897,
 'TRAVEL_AND_LOCAL': 2.335552296062281,
 'TOOLS': 8.462146000225657,
 'PERSONALIZATION': 3.317161232088458,
 'PRODUCTIVITY': 3.8925871601038025,
 'PARENTING': 0.

In [46]:
freq_table(android_final, -4)

{'Art & Design': 0.5979916506826132,
 'Art & Design;Creativity': 0.06769716800180525,
 'Auto & Vehicles': 0.9251946293580051,
 'Beauty': 0.5979916506826132,
 'Books & Reference': 2.1437436533904997,
 'Business': 4.592124562789123,
 'Comics': 0.6092745120162473,
 'Comics;Creativity': 0.011282861333634209,
 'Communication': 3.2381812027530184,
 'Dating': 1.8616721200496444,
 'Education': 5.348076272142616,
 'Education;Creativity': 0.045131445334536835,
 'Education;Education': 0.33848584000902626,
 'Education;Pretend Play': 0.05641430666817105,
 'Education;Brain Games': 0.033848584000902626,
 'Entertainment': 6.070179397495204,
 'Entertainment;Brain Games': 0.07898002933543948,
 'Entertainment;Creativity': 0.033848584000902626,
 'Entertainment;Music & Video': 0.16924292000451313,
 'Events': 0.7108202640189552,
 'Finance': 3.7007785174320205,
 'Food & Drink': 1.241114746699763,
 'Health & Fitness': 3.0802211440821394,
 'House & Home': 0.8236488773552973,
 'Libraries & Demo': 0.936477490691

In [48]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

I also used a function, provided by Dq, that sorts the frequency table. 

We can see that among the free English apps, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. In other words, the App Store is dominated by games!

There are two columns we need to look at for Android. Category and Genre. 

In [49]:
display_table(ios_final, -5)


Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


In [50]:
display_table(android_final, 1) # Category

FAMILY : 18.898792733837304
GAME : 9.725826469592688
TOOLS : 8.462146000225657
BUSINESS : 4.592124562789123
LIFESTYLE : 3.9038700214374367
PRODUCTIVITY : 3.8925871601038025
FINANCE : 3.7007785174320205
MEDICAL : 3.5315355974275078
SPORTS : 3.396141261423897
PERSONALIZATION : 3.317161232088458
COMMUNICATION : 3.2381812027530184
HEALTH_AND_FITNESS : 3.0802211440821394
PHOTOGRAPHY : 2.944826808078529
NEWS_AND_MAGAZINES : 2.798149610741284
SOCIAL : 2.6627552747376737
TRAVEL_AND_LOCAL : 2.335552296062281
SHOPPING : 2.245289405393208
BOOKS_AND_REFERENCE : 2.1437436533904997
DATING : 1.8616721200496444
VIDEO_PLAYERS : 1.7939749520478394
MAPS_AND_NAVIGATION : 1.399074805370642
FOOD_AND_DRINK : 1.241114746699763
EDUCATION : 1.1621347173643235
ENTERTAINMENT : 0.9590432133589079
LIBRARIES_AND_DEMO : 0.9364774906916393
AUTO_AND_VEHICLES : 0.9251946293580051
HOUSE_AND_HOME : 0.8236488773552973
WEATHER : 0.8010831546880289
EVENTS : 0.7108202640189552
PARENTING : 0.6544059573507841
ART_AND_DESIGN : 0

In [51]:
display_table(android_final, -4)

Tools : 8.450863138892023
Entertainment : 6.070179397495204
Education : 5.348076272142616
Business : 4.592124562789123
Productivity : 3.8925871601038025
Lifestyle : 3.8925871601038025
Finance : 3.7007785174320205
Medical : 3.5315355974275078
Sports : 3.463838429425702
Personalization : 3.317161232088458
Communication : 3.2381812027530184
Action : 3.102786866749408
Health & Fitness : 3.0802211440821394
Photography : 2.944826808078529
News & Magazines : 2.798149610741284
Social : 2.6627552747376737
Travel & Local : 2.324269434728647
Shopping : 2.245289405393208
Books & Reference : 2.1437436533904997
Simulation : 2.042197901387792
Dating : 1.8616721200496444
Arcade : 1.8503892587160102
Video Players & Editors : 1.771409229380571
Casual : 1.7601263680469368
Maps & Navigation : 1.399074805370642
Food & Drink : 1.241114746699763
Puzzle : 1.128286133363421
Racing : 0.9928917973598104
Role Playing : 0.9364774906916393
Libraries & Demo : 0.9364774906916393
Auto & Vehicles : 0.9251946293580051
S



# average number of user ratings per app genre on the App Store:

In [71]:
genres_ios = freq_table(ios_final, -5)


for genre in genres_ios:
    total = 0
    len_genre = 0
    for app in ios_final:
        genre_app = app[-5]
        if genre_app == genre:       
            n_ratings = float(app[6])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ":", avg_n_ratings)
  

Productivity : 21028.410714285714
Weather : 52279.892857142855
Shopping : 26919.690476190477
Reference : 74942.11111111111
Finance : 31467.944444444445
Music : 57326.530303030304
Utilities : 18684.456790123455
Travel : 28243.8
Social Networking : 71548.34905660378
Sports : 23008.898550724636
Health & Fitness : 23298.015384615384
Games : 22788.6696905016
Food & Drink : 33333.92307692308
News : 21248.023255813954
Book : 39758.5
Photo & Video : 28441.54375
Entertainment : 14029.830708661417
Business : 7491.117647058823
Lifestyle : 16485.764705882353
Education : 7003.983050847458
Navigation : 86090.33333333333
Medical : 612.0
Catalogs : 4004.0


In [76]:

for app in ios_final:
    if app[-5] == 'Navigation':
        print(app[2], ':', app[6]) # print name and number of ratings

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Geocaching® : 12811
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5
CoPilot GPS – Car Navigation & Offline Maps : 3582
Google Maps - Navigation & Transit : 154911


In [77]:
categories_android = freq_table(android_final, 1)

for category in categories_android:
    total = 0
    len_category = 0
    for app in android_final:
        category_app = app[1]
        if category_app == category:            
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3697848.1731343283
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:



In [78]:

for app in android_final:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

In [79]:
under_100_m = []

for app in android_final:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

3603485.3884615386

In [33]:


print(ios_header)
print('\n')
print(android_header)


['', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [32]:
print(ios_final[1])
print('\n')
print(android_final[1])

['3', '281940292', 'WeatherBug - Local Weather, Radar, Maps, Alerts', '100524032', 'USD', '0', '188583', '2822', '3.5', '4.5', '5.0.0', '4+', 'Weather', '37', '5', '3', '1']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']
